### **5.7 - [Practica] Set up the Airflow cluster with Celery Executors and Docker**

En este video vamos a configurar y utilizar Airflow con el Celery Executor. Primero, desde tu editor de código, comprueba que estás en la carpeta airflow-materials/airflow-section-5. Si abres el archivo docker-compose-CeleryExecutor.yml podrás ver los diferentes servicios que vamos a arrancar para poder utilizar Airflow con el Celery Executor. Aquí, el primer servicio corresponde al sistema broker de mensajes (message broker system) requerido por Celery del que los nodos worker extraerán las tareas para ejecutarlas. En nuestro caso, el sistema broker será Redis, que es un almacén de estructuras de datos en memoria de código abierto (open source in-memory data structure store) bastante fácil de usar e instalar. El comando de abajo se ejecutará al inicio del contenedor para tener un servidor redis configurado con la contraseña "redispass". De nuevo, siéntete libre de usar RabbitMQ si quieres, sólo tendrás que cambiar la imagen Docker aquí, así como algunos parámetros de configuración en airflow.cfg. Si quieres un video sobre esto, no dudes en contactarme en la sección Q/A. El siguiente servicio por arrancar es la metadata database de Airflow con Postgres. Mantenemos la misma configuración que con el Local Executor. Fíjate que descomenta las líneas aquí para que incluso después de reiniciar el contenedor docker, puedas seguir recuperando tus datos ya que se persistirán en disco. En nuestro ejemplo no vamos a persistir los datos, pero siéntete libre de hacerlo si quieres para tus propios experimentos. A continuación, una vez que la metadata database, así como el sistema de mensajería (message broker system) se están ejecutando, se inicia el web server de Airflow. Si echas un vistazo a las variables de entorno, el EXECUTOR se establece en Celery y hay otra variable llamada FERNET_KEY con un valor extraño. No te voy a explicar lo que es una FERNET_KEY ya que lo veremos más adelante en la sección sobre la seguridad de Airflow. Pero de momento ten en cuenta que esta clave sirve para encriptar tus conexiones y que no te puedan robar las contraseñas. A continuación, tenemos los volúmenes con el fin de vincular la carpeta "dags", así como el archivo airflow.cfg de nuestra máquina a la misma carpeta y archivo dentro del contenedor docker.  

<center><img src="https://i.postimg.cc/tRDg303D/a817.png"></center>
<center><img src="https://i.postimg.cc/653pzHzK/a818.png"></center>

A continuación, tenemos un nuevo servicio llamado flower. Celery Flower es básicamente una herramienta para monitorizar las tareas y workers de Celery. Es una aplicación basada en web y nos permite ver el progreso de las tareas, los detalles y el estado de los workers.  Como Flower está integrado con Airflow podemos iniciarlo con el comando "airflow flower". Por eso la imagen Docker es Airflow. A continuación, tenemos el servicio iniciando el scheduler de Airflow. Al contrario que con el LocalExecutor, esta vez el scheduler y el web server se ejecutan en dos contenedores docker distintos. Si usted se está preguntando por qué con el LocalExecutor estábamos ejecutando tanto el web server como el scheduler dentro del mismo contenedor docker, en realidad no hay ninguna razón en particular y es una elección de la comunidad que ha creado esta imagen Docker.  Siempre y cuando el web server y el scheduler estén ubicados en la misma máquina por razones de rendimiento.  Nótese que los volúmenes tienen que coincidir con los volúmenes dados para el servicio web server así como para el servicio worker como veremos en un minuto.  Esta es la primera cosa a tener en cuenta. Si vemos los diferentes contenedores como diferentes máquinas, cuando estás usando Celery Executors tienes que instalar Airflow en cada uno de los nodos de tu cluster.  Después de haber ejecutado el comando "airflow init db", cada nodo tendrá su propio archivo de configuración airflow.cfg, así como su propia carpeta "dags", que no es lo que queremos.  Nos gustaría que los ajustes de configuración fueran homogéneos en todo el clúster y compartieran la misma carpeta "dags".  Es por eso que establecemos los mismos volúmenes, así como las mismas variables de entorno para todos los contenedores docker que ejecutan Airflow. Para mantener las mismas configuraciones y DAGs en todo nuestro cluster.  Voy a mostrar los problemas que se pueden obtener mediante la adición de un nodo adicional al clúster sin utilizar Docker compose, como un nuevo nodo worker.  El último servicio a ejecutar es el worker correspondiente a donde se crearán los procesos celery para ejecutar las tareas sacadas de las colas de forma controlada.  

<center><img src="https://i.postimg.cc/65m5Pd1v/a819.png"></center>
<center><img src="https://i.postimg.cc/Lsd49mS4/a820.png"></center>
<center><img src="https://i.postimg.cc/vmWYDC8r/a821.png"></center>

Muy bien, ya hemos visto cómo se va a montar y configurar el cluster usando docker compose, vamos a arrancarlo desde la terminal. Desde tu terminal, comprueba que no tienes ningún contenedor docker ejecutando Airflow con el comando "docker ps".  Deberías tener la misma pantalla que yo. Luego verifica que estás en la carpeta airflow- materials/airflow-section-5 con los siguientes archivos y carpetas cuando escribes "ls". Muy bien. Escribe el comando **`"docker-compose -f docker-compose-CeleryExecutor.yml up -d"`**.  Enter.  Si es la primera vez que utilizas Redis, Docker descargará la imagen docker y luego iniciará el contenedor.  Este proceso puede ser bastante largo, así que voy a pausar el vídeo y volveré cuando haya terminado. 

<center><img src="https://i.postimg.cc/tTpX6D7L/a822.png"></center>

Al final deberías obtener esas diferentes líneas mostrando que los diferentes servicios han sido creados. 

<center><img src="https://i.postimg.cc/gj0mSJVs/a823.png"></center>

Si escribe "docker ps", debería obtener una salida equivalente con los contenedores worker, scheduler, web server, flower, postgres y redis en ejecución. 

<center><img src="https://i.postimg.cc/PxydQZwN/a824.png"></center>

Abra su navegador web y vaya a localhost :8080 para comprobar que Airflow se está ejecutando.  Ok.  Abra una nueva pestaña y escriba localhost:5555.  Enter. Usted debe aterrizar en el Flower dashboard con un worker con el estado "Online". Perfecto.  

<center><img src="https://i.postimg.cc/rwRF2KvY/a825.png"></center>

Antes de pasar al siguiente vídeo me gustaría darte un comando muy útil.  Vuelve a tu terminal. Si tienes algún error durante el proceso, puedes comprobar los logs de tus contenedores docker escribiendo "docker logs" y copiando y pegando el id del contenedor que quieras.  Tomemos por ejemplo el nodo worker.  Copiamos el id, lo pegamos y le damos a enter.  

<center><img src="https://i.postimg.cc/PJFhBjKj/a826.png"></center>

Como puedes ver hemos obtenido los logs del worker.  

<center><img src="https://i.postimg.cc/nVdHs4xP/a827.png"></center>

Por último, fíjate que puedes usar la opción -f . Así que escribe "docker logs -f", el id del web server por ejemplo.  Enter. Y ahora puedes ver los logs continuamente. 

<center><img src="https://i.postimg.cc/MKvz1nKK/a828.png"></center>

Para detener el comando, pulsa CTRL + C. Muy bien. Hemos configurado y puesto en marcha el clúster Airflow con el CeleryExecutor y un worker, vamos a ejecutar algunas tareas en el siguiente vídeo.